# Managing State in Autogen: Saving and Loading Agent Conversations

## Setup and Requirements

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [2]:
import json
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

## Introduction to State Management
In multi-agent applications, preserving the context and history of conversations is crucial. State management in Autogen allows us to save and restore agent memories, team configurations, and conversation histories.

## Managing Individual Agent State

In [3]:
# Create a movie critic agent
critic_agent = AssistantAgent(
    name="critic_agent",
    system_message="You are a passionate movie critic who analyzes films deeply",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Have a conversation about a movie
response = await critic_agent.on_messages(
    [TextMessage(content="Review the cinematography of Blade Runner 2049", source="user")], 
    CancellationToken()
)

# Save the agent's state after the review
agent_state = await critic_agent.save_state()

# Create a new instance of the critic agent
new_critic_agent = AssistantAgent(
    name="critic_agent",
    system_message="You are a passionate movie critic who analyzes films deeply",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Load the previous conversation state
await new_critic_agent.load_state(agent_state)

# Ask about the previous review
response = await new_critic_agent.on_messages(
    [TextMessage(content="What were the main points of your previous cinematography analysis?", source="user")], 
    CancellationToken()
)

In [4]:
response

Response(chat_message=TextMessage(source='critic_agent', models_usage=RequestUsage(prompt_tokens=730, completion_tokens=259), content='The main points of the cinematography analysis of "Blade Runner 2049" include:\n\n1. **Masterful Atmosphere Creation**: Roger Deakins\' cinematography crafts immersive environments that reflect the film\'s themes of isolation and existential inquiry, particularly through vast and oppressive urban landscapes.\n\n2. **Dynamic Use of Color**: The film\'s palette showcases a range of hues from muted earth tones to vibrant neon colors, establishing emotional depth and mirroring the film’s exploration of artificiality versus authenticity.\n\n3. **Meticulous Framing**: Every shot is composed with great care, achieving harmonious or tense visual narratives that underscore the duality of human and replicant experiences.\n\n4. **Innovative Lighting Techniques**: Deakins utilizes a blend of natural and artificial lighting to sculpt mood and atmosphere, creating ef

## Managing Team State

In [5]:
# Create a film analysis team
critic_agent = AssistantAgent(
    name="critic_agent",
    system_message="You are a passionate movie critic who analyzes films deeply",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4-0125-preview",
    ),
)

film_team = RoundRobinGroupChat(
    [critic_agent], 
    termination_condition=MaxMessageTermination(max_messages=3)
)

# Run an analysis session
stream = film_team.run_stream(
    task="Analyze the themes of isolation in Taxi Driver"
)
await Console(stream)

# Save the team's state
team_state = await film_team.save_state()

# Create a new team instance
new_film_team = RoundRobinGroupChat(
    [critic_agent], 
    termination_condition=MaxMessageTermination(max_messages=3)
)

# Load the previous analysis session
await new_film_team.load_state(team_state)

# Ask about the previous analysis
stream = new_film_team.run_stream(
    task="What was your main conclusion about isolation in the previous analysis?"
)
await Console(stream)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was your main conclusion about isolation in the previous analysis?', type='TextMessage'), TextMessage(source='critic_agent', models_usage=RequestUsage(prompt_tokens=780, completion_tokens=167), content='In the analysis of "Taxi Driver," the main conclusion about isolation is that it is portrayed as a deeply pervasive and destructive force that significantly impacts the protagonist, Travis Bickle. Isolation is not only a personal affliction for Travis but also a symptom of the broader societal and urban condition. This profound sense of alienation drives him into a dangerous downward spiral, culminating in violence and delusion. The film meticulously dissects how isolation can distort an individual\'s reality, pushing them towards drastic measures in a desperate search for meaning, connection, and redemption in an indifferent world. Ultimately, "Taxi Driver" serves as a powerful exploration of the human con

## Persistent Storage

In [7]:
# Save team state to a file
def save_team_state(team_state, filename):
    with open(filename, "w") as f:
        json.dump(team_state, f)

# Load team state from a file
def load_team_state(filename):
    with open(filename, "r") as f:
        return json.load(f)

# Example usage
filename = "film_analysis_session.json"

# Save the state
save_team_state(team_state, filename)

# Create a new team and restore the state
new_team = RoundRobinGroupChat(
    [critic_agent], 
    termination_condition=MaxMessageTermination(max_messages=3)
)
saved_state = load_team_state(filename)
await new_team.load_state(saved_state)

# Continue the analysis
stream = new_team.run_stream(
    task="How does the theme of isolation connect to the visual style discussed earlier?"
)
output = await Console(stream)
output.messages[-1].content

'The use of visual style in "Taxi Driver" to articulate the theme of isolation is not only clever but integral to the film\'s enduring impact. Here are a few key ways the visual style enhances the theme:\n\n### Visual Metaphors and Symbolism\n\nThe city itself, with its blinding neon lights and shadowy alleys, serves as a visual metaphor for Travis Bickle\'s isolation. The contrast between the bright, bustling streets and the dark, empty corners mirrors Travis\'s own internal contradictions and his feelings of being both within and yet apart from the urban landscape around him. The visual motif of reflections, often seen in mirrors or through windows, further emphasizes Travis\'s fragmented identity and his disconnect from society.\n\n### Camera Techniques\n\nScorsese\'s use of camera techniques plays a crucial role in conveying Travis\'s isolation. Long, lingering shots that follow Travis through the neon-lit streets of New York at night envelop him in the city yet simultaneously unde

## Key Points to Remember

1. Agent state primarily consists of conversation history and context
2. Team state includes all member agents' states plus team-specific configurations
3. States can be serialized to JSON for persistent storage
4. Loaded states allow agents to maintain conversation context
5. State management is crucial for building stateless applications

## Common Use Cases
- Web applications requiring conversation persistence
- Long-running analysis sessions that need to be resumed
- Multi-step processes that require context preservation
- Distributed systems where state needs to be shared
- Backup and recovery of agent conversations